In [3]:
import numpy as np

from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import row, gridplot, column

output_notebook(hide_banner=True)

# Méthode des différences divisées

**Implémenter une fonction qui prend en entrée des abscisses $x_k$ et des valeurs $f(x_k)$, et qui retourne les coefficients $f[x_0,\ldots,x_k]$. On pourra utiliser :**

$$
f[x_0,\ldots,x_{n+1}]=\frac{f[x_{1},\ldots,x_{n+1}]-f[x_0,\ldots,x_{n}]}{x_{n+1}-x_0}.
$$

**en remarquant que $f[x_k]=f(x_k)$ pour tout $k$.**

In [26]:
def compute_divided_diff_coef(x, y):
    
    # Voir le commentaire en dessous
    
    n = len(x)
    coef = np.zeros(n)
    D = {}
    
    # Remplissage de D
    for k in range(n) :
        D[(k,)] = y[k]
    for k in range(1,n+1) :
        for i in range(0,n-k) :
            t = tuple([ j for j in range(i,i+k+1)])
            D[t] = ( D[t[1:]] - D[t[:-1]] )/(x[i+k] - x[i])
    
    # Remplissage de coef
    for k in range(n) :
        t = tuple([ j for j in range(0,k+1)])
        coef[k] = D[t]
    
    return coef

**Commentaire :**  
Bien qu il soit possible de calculer les différences divisées de facon récursive avec un code plus court, la récursivité ménera à calculer certains termes plusieurs fois, ce qui encombre la mémoire si on a un grand nombre de points (ce qui sera à priori le cas), et sa complexité est exponentielle. C'est pourquoi j'ai préféré utiliser un dictionnaire où seront stockées les valeurs calculées, ainsi chaque terme sera calculé une seule fois, et la complexité de ma fonction est $O(n^2)$

**Implémenter une fonction qui prend en entrée des abscisses $x_k$ les coefficients $f[x_0,\ldots,x_k]$ et un vecteurs de points de $[a,b]$, et qui renvoie la valeur de $P_n(f)$ en ces différents points. On pourra utiliser :**

$$
P_n(f)(x)=\sum_{k=0}^n f[x_0,\ldots,x_k] \Pi_k(x),\quad \forall~x\in[a,b].
$$

**et un algorithme d'évaluation de type Horner.**

In [27]:
def poly_interp(coef, xk, x):
    n = len(coef) - 1
    p = np.zeros(len(x))
    for i in range(len(x)) :
        p[i] = coef[-1]
        for j in range(1,n+1) :
            p[i] = p[i]*(x[i] - xk[n-j]) + coef[n-j]
    return p

**Générer aléatoirement des abscisses $x_k\in[-1,1]$ et des valeurs $y_k\in\mathbb{R}$, $0\leq k\leq 11$, et utiliser les fonctions construites aux deux questions précédentes pour tracer l'unique polynôme $P_n$ de degré $n$ vérifiant $P_n(x_k)=y_k$ pour tout $k$. Commenter.**

In [60]:
a = -1
b = 1

n = 10
xk = a + (b-a)*np.random.rand(n+1)
yk = 11*(np.random.randn(n+1))

# points auxquels p sera évalués pour le tracé
x = np.linspace(a,b,1000)

coef = compute_divided_diff_coef(xk, yk)
p = poly_interp(coef, xk, x)

# Tracer les points (xk, yk) et le polynome p
fig = figure(width=490, height=300)
fig.x(xk,yk,size=10, color="red", legend="pts aléatoires")
fig.line(x,p, legend="polynome p")


#######################################################################################
# Approximation de la fonction sinus
asin = -5
bsin = 5

n = 10
xk_sin = asin + (bsin-asin)*np.random.rand(n+1)
sk = np.sin(xk_sin)
xsin = np.linspace(asin,bsin,1000)
ysin = np.sin(xsin)
coef_sin = compute_divided_diff_coef(xk_sin, sk)
psin = poly_interp(coef_sin, xk_sin, xsin)


fig2 = figure(width=490, height=300)
fig2.x(xk_sin,sk,size=10, color="red", legend="pts aléatoires")
fig2.line(xsin,ysin, legend="sin", color="orange")
fig2.line(xsin,psin, legend="P(sin)")


print("figure gauche : polynôme interpolant des points aléatoires dans [-1,1]\nfigure droite : polynôme interpolant des points aléatoires sur la courbe se sin dans [-5,5]")

show(row(fig,fig2))

figure gauche : polynôme interpolant des points aléatoires dans [-1,1]
figure droite : polynôme interpolant des points aléatoires sur la courbe se sin dans [-5,5]


**Commentaire :**  
On remarque qu'au bords de la 1ère figure, le comportement du polynôme n'est pas controlé par les points d'interpolation choisis, il varie très rapidement. C'est le phénomène de Runge.

Par contre, sur la 2ème figure, même avec des points aléatoires, le polynôme produit avec une très bonne approximation la fonction sinus. Ceci est dû à la régularité de sin et au fait que toutes des dérivées sont majorées en valeur absolue par 1.

# Phénomène de Runge

**Considérer la fonction :**

$$
f(x) =\frac{1}{0.1 +x^2}
$$

**sur l'intervalle $[-1,1]$, et tracer le polynôme d'interpolation $P_n(f)$ pour différentes valeurs de $n$ et différents choix de $x_k$, ainsi que l'erreur entre $f$ et $P_n(f)$. Commenter.**

In [33]:
def f(x):
    return 1/(0.1+x*x)


## Pour pouvoir appliquer f directement aux vecteurs
f_vect = np.vectorize(f)

In [53]:
# En utilisant les fonctions de la partie précedente, tracer les points (xk, yk) et le polynome p 
# pour différentes valeurs de n et différents choix de xk (uniforme et Chebyshev).
# Tracer également l'erreur entre f et p.

a = -1
b = 1
n = 20
x = np.linspace(a,b,1000)
y = f_vect(x)

##########################################################################################
# Points aléatoires sur [a,b]
xk_al = a + (b-a)*np.random.rand(n+1)
yk_al = f_vect(xk_al)

coef_al = compute_divided_diff_coef(xk_al, yk_al)
p_al = poly_interp(coef_al, xk_al, x)

err_al = abs(p_al - y)

fig1 = figure(width=490, height=300)
fig1.x(xk_al,yk_al,size=10, color="red", legend="pts aléatoires")
fig1.line(x,y, color = "orange")
fig1.line(x,p_al, legend="P(f)")

fig1b = figure(width=490, height=300, y_axis_type="log")
fig1b.line(x,err_al, legend="|f-P(f)|")


##########################################################################################
# Points equidistants sur [a,b]
xk_eq = np.linspace(a,b,n)
yk_eq = f_vect(xk_eq)

coef_eq = compute_divided_diff_coef(xk_eq, yk_eq)
p_eq = poly_interp(coef_eq, xk_eq, x)

err_eq = abs(p_eq - y)

fig2 = figure(width=490, height=300)
fig2.x(xk_eq,yk_eq,size=10, color="red", legend="pts equidistants")
fig2.line(x,y, color = "orange")
fig2.line(x,p_eq, legend="P(f)")

fig2b = figure(width=490, height=300, y_axis_type="log")
fig2b.line(x,err_eq, legend="|f-P(f)|")


##########################################################################################
# Points de Tchebychef sur [a,b]
xk_tbf = np.array([(a + (b-a)/2*(1 + np.cos((2*k+1)/(2*n+1)*np.pi))) for k in range(n+1)])
yk_tbf = f_vect(xk_tbf)


coef_tbf = compute_divided_diff_coef(xk_tbf, yk_tbf)
p_tbf = poly_interp(coef_tbf, xk_tbf, x)

err_tbf = abs(p_tbf - y)

fig3 = figure(width=490, height=300)
fig3.x(xk_tbf,yk_tbf,size=10, color="red", legend="pts de Tchebychef")
fig3.line(x,y, color = "orange")
fig3.line(x,p_tbf, legend="P(f)")

fig3b = figure(width=490, height=300, y_axis_type="log")
fig3b.line(x,err_tbf, legend="|f-P(f)|")



grid = gridplot([[fig1, fig1b], [fig2, fig2b], [fig3, fig3b]])
show(grid)

**Commentaire :**  
La fonction $f$ est lipschitzienne sur $[a,b]$, en effet :
$$\forall x,y \in [a,b] : |f(x)-f(y)|=\frac{|0.1+y^2-0.1-x^2|}{(0.1+x^2)(0.1+y^2)}
=\frac{|x+y||x-y|}{(0.1+x^2)(0.1+y^2)} \leq \frac{2max(|a|,|b|)}{0.1^2}|x-y| $$

Le résultat de la 2ème question de l'exercice 3 nous assure l'existence d'une constante $C_f$ telle que
$$||P_n(f) - f||_{\infty} \leq C_f\frac{1+\Lambda_n}{\sqrt{n}} $$

* Pour des points equidistants : $\Lambda_n \sim \frac{2^{n+1}}{en\ln(n)}$, et donc rien ne garantit la convergence de $(P_n(f))_n$ vers $f$.  
* Pour des points de Tchebychef : $\Lambda_n \sim \frac{2}{\pi}\ln(n)$, on a donc convergence vers 0 de $\frac{1+\Lambda_n}{\sqrt{n}}$, et par suite $(P_n(f))_n$ converge vers $f$.


# Interpolation de Hermite

**Implémenter la méthode et tester sur l'exemple suivant :**

$$
f(x) =\frac{1}{0.1 +x^2}
$$

In [55]:
# Implémenter la méthode
# la fonction interp_Hermite retourne le vecteur des valeurs du polynome évalué aux points de x

def denom2_dl(xk) :
    denom = np.ones(len(xk))
    dl = np.zeros(len(xk))
    for j in range(len(xk)) :
        for i in range(len(xk)) :
            if i!=j :
                denom[j]*=(xk[j]-xk[i])
                dl[j]+=1/(xk[j]-xk[i])
    return denom*denom, dl

def scinde_carre(xk,k,x) :
    S = np.ones(len(x))
    for j in range(len(xk)) :
        if k!=j : S*= (x-xk[j])
    return S*S

def interp_Hermite(xk,yk,dyk,x) :
    denom2,dl = denom2_dl(xk)
    Q = np.zeros(len(x))
    H0 = np.zeros(len(x))
    H1 = np.zeros(len(x))
    for k in range(len(xk)) :
        scinde2 = scinde_carre(xk,k,x)
        l2 = scinde2/denom2[k]
        H0 = (1-2*dl[k]*(x-xk[k]))*l2
        H1 = (x-xk[k])*l2
        Q += yk[k]*H0 + dyk[k]*H1
    return Q

In [56]:
def f(x):
    return 1/(0.1+x*x)

def df(x) :
    return -2*x/((0.1 +x*x)**2)

## Pour pouvoir appliquer f et df directement aux vecteurs
f_vect = np.vectorize(f)
df_vect = np.vectorize(df)

In [57]:
# Tracer les points (xk, yk) et le polynome d'interpolation de Hermite h, 
# comparer avec le polynome d'interpolation de Lagrange p
a = -1
b = 1
n = 15
x = np.linspace(a,b,1000)
y = f_vect(x)
xk = np.linspace(a,b,n)
yk = f_vect(xk)
dyk = df_vect(xk)
xkT = np.array([(a + (b-a)/2*(1 + np.cos((2*k+1)/(2*n+1)*np.pi))) for k in range(n+1)])
ykT = f_vect(xkT)
dykT = df_vect(xkT)

#################################################################################
# Interpolation de Hermite avec points equidistants
QE = interp_Hermite(xk,yk,dyk,x)
errQE = abs(QE - y)

fig1 = figure(width=490, height=300)
fig1.x(xk,yk,size=10, color="red", legend="pts equidistants")
fig1.line(x,y, color="orange")
fig1.line(x,QE, legend="Q(f)")

fig1b = figure(width=490, height=300,y_axis_type="log")
fig1b.line(x,errQE, legend="|f-Q(f)|")


#################################################################################
# Interpolation de Lagrange avec points equidistants
coefE = compute_divided_diff_coef(xk, yk)
PE = poly_interp(coefE, xk, x)

errPE = abs(PE - y)

fig2 = figure(width=490, height=300)
fig2.x(xk,yk,size=10, color="red", legend="pts equidistants")
fig2.line(x,y, color = "orange")
fig2.line(x,PE, legend="P(f)")

fig2b = figure(width=490, height=300, y_axis_type="log")
fig2b.line(x,errPE, legend="|f-P(f)|")

#################################################################################
# Interpolation de Hermite avec points de Tchebychef
QT = interp_Hermite(xkT,ykT,dykT,x)
errQT = abs(QT - y)

fig3 = figure(width=490, height=300)
fig3.x(xkT,ykT,size=10, color="red", legend="pts de Tchebychef")
fig3.line(x,y, color="orange")
fig3.line(x,QT, legend="Q(f)")

fig3b = figure(width=490, height=300, y_axis_type="log")
fig3b.line(x,errQT, legend="|f-Q(f)|")


#################################################################################
# Interpolation de Lagrange avec points de Tchebychef
coefT = compute_divided_diff_coef(xkT, ykT)
PT = poly_interp(coefT, xkT, x)

errPT = abs(PT - y)

fig4 = figure(width=490, height=300)
fig4.x(xkT,ykT,size=10, color="red", legend="pts de Tchebychef")
fig4.line(x,y, color = "orange")
fig4.line(x,PT, legend="P(f)")

fig4b = figure(width=490, height=300, y_axis_type="log")
fig4b.line(x,errPT, legend="|f-P(f)|")

#################################################################################

grid = gridplot([[fig1,fig1b],[fig2,fig2b],[fig3,fig3b],[fig4,fig4b]])
show(grid)

**Commentaire :**  
L'approximation par les polynômes de Hermite est meilleure que celle par les polnômes de Lagrange.    
En testant avec des petites valeurs de $n$, on remarque que le polynôme d'Hermite s'approche plus de la fonction $f$. Ce qui est normal puisqu'on utilise plus d'informations. Toutefois, pour $n\geq 15$, le phénomène de Runge aux bords est plus viloent pour les polynômes d'Hermite lorsqu'on choisit une subdivision par des points equidistants.

La majoration de l'erreur entre $f$ et $Q(f)$ établie dans la question 3 du $1^{er}$ exercice de la PC3 montre que les points de Tchebychef donneront encore une fois la meilleure approximation parmi toutes les subdivisions de $[a,b]$ de taille $n$ :
$$ ||f-Q_n(f)|| \leq \frac{||\Pi_{n+1}||_{\infty}}{(2n+2)!}||f^{(2n+2)}||_{\infty}$$  

car les points de Tchebychef minimisent la quantité $||\Pi_{n+1}||_{\infty}$

# Krigeage

**On considère les 13 points $(x_k,y_k)$ suivants :**

$$(-3,0.5),\ (-2.5,0),\ (-2,2),\ (-1.5,1.5),\ (-1,4),\ (-0.5,2),\ (0,1.5),\ (0.5,3),\ (1,0),\ (1.5,2),\ (2,1),\ (2.5,2.5),\ (3,3)$$

**Implémenter $G$, $A$ et $Y$, puis résoudre le système linéaire et tracer la fonction $u$ obtenue pour $g(x)=x$ et $g(x)=x^3$. Comparer avec le polynôme d'interpolation de Lagrange. Commenter les résultats obtenus.**

In [59]:
def g(x):
    return x**3

g_vect = np.vectorize(g)

In [60]:
xk = np.linspace(-3,3,13)
yk = np.array([0.5,0,2,1.5,4,2,1.5,3,0,2,1,2.5,3])
x = np.linspace(-3,3,1000)

In [61]:
A = np.ones((13,2))
for i in range(13) :
    A[i][1] = xk[i]
# ... 

G = np.zeros((13,13))
for i in range (13) :
    G[i][i] = g(0)
    for j in range(i) :
        G[i][j] = G[j][i] = g(abs(xk[i]-xk[j]))
# ...

M = np.concatenate((np.concatenate((G,np.transpose(A))),np.concatenate((A,np.zeros((2,2))))),axis=1)

Y = np.hstack(( yk, np.array([0,0]) ))
# ...

# Résolution de M S = Y
S = np.linalg.solve(M,Y)

# Evaluation de la fonction de krigeage u en un vecteur de points x
def u_kri(x, S, xk):
    u = S[-2] + S[-1]*x
    for k in range(len(xk)) :
        u += S[k]*g_vect(abs(x-xk[k]))
    return u

# Fonction de krigeage
u = u_kri(x, S, xk)

fig = figure(width=490, height=300)
fig.x(xk,yk,size=10, color="red", legend="pts d'interpolation")
fig.line(x,u, legend="u(x)")

# Polynôme de Lagrange
coef = compute_divided_diff_coef(xk, yk)
p = poly_interp(coef, xk, x)
fig.line(x,p, color="orange", legend="P(x)")

# Erreur
err = abs(p-u)
fig2 = figure(width=490, height=300,y_axis_type="log")
fig2.line(x,err, legend="|P(x)-u(x)|")

show(row(fig,fig2))

**Commentaire :**  
La fonction de krigeage obtenue est plus régulière pour $g(x)=x^3$ (courbe lisse) que pour $g(x)=x$ (courbe par lignes brisées).  
Ensuite, pour comparer la fonction de krigeage avec le polynôme de Lagrange, j'ai tracé à droite la courbe de l'erreur entre les deux : les deux fonctions sont assez proches sur l'intervalle $[-1.4, 1]$, mais aux bords, le phénomène de Runge fait que le polynôme $P$ varie très rapidement, alors que $u$ ne s'éloigne pas des points d'interpolation.  
Ceci nous laisse penser que pour une fonction ayant des variations lentes, comme $f(x) = \frac{1}{0.1 + x^2}$, la fonction de krigeage fournirait une meilleure approximation que le polynôme de Lagrange en interpolant aux mêmes points.

In [62]:
def f(x):
    return 1/(0.1+x*x)


## Pour pouvoir appliquer f directement aux vecteurs
f_vect = np.vectorize(f)

In [69]:
a = -1
b = 1
n = 20
x = np.linspace(a,b,1000)
y = f_vect(x)

## Choisir ici le type de subdivision : 

#xk = a + (b-a)*np.random.rand(n)
#xk = np.linspace(a,b,n)
xk = np.array([(a + (b-a)/2*(1 + np.cos((2*k+1)/(2*n+1)*np.pi))) for k in range(n+1)])
yk = f_vect(xk)

#################################################################################
# Polynôme de Lagrange
coef = compute_divided_diff_coef(xk, yk)
p = poly_interp(coef, xk, x)
errp = abs(p - y)

fig1 = figure(width=490, height=300)
fig1.x(xk,yk,size=10, color="red", legend="pts d'interpolation")
fig1.line(x,y, color = "orange")
fig1.line(x,p, legend="P(f)")

fig1b = figure(width=490, height=300, y_axis_type="log")
fig1b.line(x,errp, legend="|f-P(f)|")

#################################################################################
# Fonction de krigeage
A = np.ones((len(xk),2))
for i in range(len(xk)) :
    A[i][1] = xk[i]

G = np.zeros((len(xk),len(xk)))
for i in range (len(xk)) :
    G[i][i] = g(0)
    for j in range(i) :
        G[i][j] = G[j][i] = g(abs(xk[i]-xk[j]))

M = np.concatenate((np.concatenate((G,np.transpose(A))),np.concatenate((A,np.zeros((2,2))))),axis=1)
Y = np.hstack(( yk, np.array([0,0]) ))
S = np.linalg.solve(M,Y)

u = u_kri(x, S, xk)
erru = abs(u-y)
fig2 = figure(width=490, height=300)
fig2.x(xk,yk,size=10, color="red", legend="pts d'interpolation")
fig2.line(x,y, color = "orange")
fig2.line(x,u, legend="u(f)")

fig2b = figure(width=490, height=300, y_axis_type="log")
fig2b.line(x,erru, legend="|f-u(f)|")

#################################################################################

grid = gridplot([[fig1,fig1b],[fig2,fig2b]])
show(grid)

**Commentaire :**  
En testant les 2 méthodes d'interpolation (avec $g(x)=x^3$ pour le krigeage), avec différents types de subdivisions de l'intervalle $[a,b]$, on remarque que la fonction de krigeage donne une meilleure approximation de $f(x)$, même en choisissant une subdivision par les points de Tchebychef.